In [ ]:
## 设置数据库 & 定义数据模型
    
load './wtnb/wtnb.rb'
load './wtnb/wtmodel.rb'

IRuby.display IRuby.html(include_js)
IRuby.display IRuby.html(include_css)
nil

In [6]:
## 引入人群  & 商品

group = Store["group_marriage"]
product = Store["product_shortlist"]
nil

In [7]:
def spu_history(spu,date)
    self.history = date
    
    product = ProductSpu.where(spu:spu).first
    
    if product.nil? then
        return [0,0,0,0,0]
        
    else

        #总库存
        total_stock = product.warehouse.size 

        #售出数量
        sold_stock = product.warehouse.sold.size

        #可用库存
        remain_stock = product.warehouse.unsold.size

        # 在单比
        in_order_rate = number_to_percentage(100- product.warehouse.inorder.size.to_f*100 / remain_stock )

        # 在途比
        stock = product.warehouse.unsold.includes(:item).map(&->(x){
            begin
                x.item.keep.last.orders.onroad
            rescue NoMethodError
                nil
            end
        })
        in_trip_rate = number_to_percentage(stock.count(true).to_f/stock.count*100)

        return [total_stock,sold_stock,remain_stock,in_order_rate,in_trip_rate]
    end
end
    
def spu_detail(spu,date_start,date_end)
    total_stock = {}
    sold_stock = {}
    remain_stock = {}
    in_order_rate = {}
    in_trip_rate = {}
    while (date_start < date_end) do
        date= date_start.to_s(:db)[0,10].gsub(/-/,"")
        d1,d2,d3,d4,d5  = spu_history(spu,date)      
        total_stock[date]=d1
        sold_stock[date]=d2
        remain_stock[date]=d3
        in_order_rate[date]=d4
        in_trip_rate[date]=d5
        
        date_start += 1.days
    end
    self.history = nil
    return [total_stock,sold_stock,remain_stock,in_order_rate,in_trip_rate]
end

product_list = []
    
date_start = Time.zone.parse( (Time.now()-1.month).to_s(:db)[0,10])
date_end = Time.zone.parse( Time.now().to_s(:db)[0,10] )

product.each do |x|
    spu = x.spu
    
    timebegin = Time.now()
    total_stock,sold_stock,remain_stock,in_order_rate,in_trip_rate = spu_detail(spu,date_start,date_end)
    timeend = Time.now()
    puts "Time: #{humanize(timeend-timebegin)}"

    product = {:name => spu.to_s + " " + ProductSpu.where(spu:spu).first.product_name,
          :picture => (ProductPicture.where(spu:spu).first_pic.map {|y| y.url }.first or ProductPicture.where(spu:spu).map {|y| y.url }.first), 
          :purchase_time => ProductSpu.where(spu:spu).first.warehouse.map do |x| x.create_time.to_s(:db)[0,10] end.uniq.size,
          :purchase_price => ProductSpu.where(spu:spu).first.purchase_price,
          :sold_amount => ProductSpu.where(spu:spu).first.item.orders.finish_purchase.item.keep.sold.where(spu:spu).map do |x| x.price_real.to_f end.inject(0){|sum,x| sum + x },
          :purchase_rate => ratio(ProductSpu.where(spu:spu).first.item.orders.finish_purchase.item.keep.where(spu:spu).sold.size , ProductSpu.where(spu:spu).first.item.orders.finish_purchase.item.keep.where(spu:spu).size),
          :total_stock => total_stock,
          :sold_stock => sold_stock,
          :remain_stock => remain_stock,
          :in_order_rate =>in_order_rate,
          :in_trip_rate =>in_trip_rate} 

    product_list.append([
     ratio( ratio(product[:sold_amount] , product[:sold_stock].to_a.last[1]),product[:purchase_price]),
     ratio(product[:sold_stock].to_a.last[1] ,product[:total_stock].to_a.last[1].to_f),
     ratio(product[:sold_amount],(product[:purchase_price]*product[:total_stock].to_a.last[1])),
     number_to_percentage(product[:purchase_rate]*100),
     product[:picture],
     product[:total_stock].to_a.last[1],
     product[:sold_stock].to_a.last[1],
     product[:remain_stock].to_a.last[1],
     product[:purchase_time],
     product[:purchase_price],
     product[:purchase_price]*product[:total_stock].to_a.last[1],
     ratio(product[:sold_amount] , product[:sold_stock].to_a.last[1]),
     product[:sold_amount].round,
     [{name:"总库存",data:product[:total_stock]}, {name:"售出数量",data:product[:sold_stock]}, {name:"可用库存",data:product[:remain_stock]}],
     [{name:"在单比",data:product[:in_order_rate]}, {name:"在途比",data:product[:in_trip_rate]}],
     product[:name],         
    ])

end

nil

Time: 16 seconds
Time: 17 seconds
Time: 15 seconds
Time: 19 seconds
Time: 13 seconds
Time: 16 seconds
Time: 23 seconds
Time: 25 seconds
Time: 14 seconds
Time: 17 seconds
Time: 31 seconds
Time: 38 seconds
Time: 25 seconds
Time: 20 seconds
Time: 20 seconds
Time: 24 seconds
Time: 27 seconds
Time: 23 seconds
Time: 46 seconds
Time: 22 seconds
Time: 19 seconds
Time: 15 seconds
Time: 34 seconds
Time: 17 seconds
Time: 10 seconds
Time: 8 seconds
Time: 15 seconds


In [8]:
puts_display render("主推品一览表",{:product_list=>product_list})
puts_to_file(render("主推品一览表",{:product_list=>product_list}),"output/商品候选一览表.html")

"\n<div id=\"app_JUrNxSL6b6\">\n<el-container>\n\n  <el-container>\n\n  <el-header>\n    <el-row :gutter=\"24\">\n      <el-col :span=\"4\">\n        <el-button icon=\"el-icon-caret-left\" circle v-on:click=\"left\"></el-button>\n        <el-button icon=\"el-icon-caret-right\" circle v-on:click=\"right\"></el-button>\n      </el-col>\n\n      <el-col :span=\"20\">\n        <h1>[{{ cur_product+1 }}] {{ product_list[cur_product][15] }}</h1>\n      </el-col>\n    </el-row>\n  </el-header>\n  <el-main>\n    <el-divider></el-divider>\n    <el-row>\n      <el-col :span=\"12\">\n        <el-button type=\"primary\" round>加价倍数: {{ product_list[cur_product][0] }}</el-button>\n        <el-button type=\"primary\" round>售罄率: {{ product_list[cur_product][1] }}</el-button>\n        <el-button type=\"success\" round>回本率: {{ product_list[cur_product][2] }}</el-button>\n        <el-button type=\"success\" round>购买转化率: {{ product_list[cur_product][3] }}</el-button>\n      </el-col>\n    </el-row>\n    <el-row :gutter=\"20\">\n      <el-col :span=\"8\">\n        <img style=\"width: 100%;\" v-bind:src=\"product_list[cur_product][4]\">\n      </el-col>\n      <el-col :span=\"16\">\n        <line-chart :data=\"chart1\"></line-chart>\n        <line-chart :data=\"chart2\"></line-chart>\n        <el-row>\n          <el-button round>采购件数: {{ product_list[cur_product][5] }}</el-button>\n          <el-button round>销售件数: {{ product_list[cur_product][6] }}</el-button>\n          <el-button round>库存件数: {{ product_list[cur_product][7] }}</el-button>\n          <el-button round>补货次数: {{ product_list[cur_product][8] }}</el-button>\n          <el-button round>采购价格: {{ product_list[cur_product][9] }}</el-button>\n          <el-button round>采购成本: {{ product_list[cur_product][10] }}</el-button>\n          <el-button round>销售价格: {{ product_list[cur_product][11] }}</el-button>\n          <el-button round>销售总额: {{ product_list[cur_product][12] }}</el-button>\n        </el-row>\n      </el-col>\n    </el-row>\n    </el-main>\n  </el-container>\n</el-container>\n</div>\n\n\n\n<script>\n  Vue.use(VueChartkick.use(Chart))\n\n  app = new Vue({\n    el: '#app_JUrNxSL6b6',\n    data: {\n      product_list: [[\"1.38\",0.53,\"0.74\",\"22.000%\",\"https://image.wetry.shop/product/spu/10108/10108_a.jpg\",15,8,6,1,\"88.0\",\"1320.0\",121.61,973,[{\"name\":\"总库存\",\"data\":{\"20200323\":15,\"20200324\":15,\"20200325\":15,\"20200326\":15,\"20200327\":15,\"20200328\":15,\"20200329\":15,\"20200330\":15,\"20200331\":15,\"20200401\":15,\"20200402\":15,\"20200403\":15,\"20200404\":15,\"20200405\":15,\"20200406\":15,\"20200407\":15,\"20200408\":15,\"20200409\":15,\"20200410\":15,\"20200411\":15,\"20200412\":15,\"20200413\":15,\"20200414\":15,\"20200415\":15,\"20200416\":15,\"20200417\":15,\"20200418\":15,\"20200419\":15,\"20200420\":15,\"20200421\":15,\"20200422\":15}},{\"name\":\"售出数量\",\"data\":{\"20200323\":6,\"20200324\":6,\"20200325\":6,\"20200326\":6,\"20200327\":6,\"20200328\":6,\"20200329\":6,\"20200330\":6,\"20200331\":6,\"20200401\":6,\"20200402\":6,\"20200403\":6,\"20200404\":6,\"20200405\":6,\"20200406\":6,\"20200407\":6,\"20200408\":6,\"20200409\":6,\"20200410\":6,\"20200411\":6,\"20200412\":7,\"20200413\":7,\"20200414\":7,\"20200415\":7,\"20200416\":7,\"20200417\":8,\"20200418\":8,\"20200419\":8,\"20200420\":8,\"20200421\":8,\"20200422\":8}},{\"name\":\"可用库存\",\"data\":{\"20200323\":9,\"20200324\":9,\"20200325\":9,\"20200326\":9,\"20200327\":9,\"20200328\":9,\"20200329\":9,\"20200330\":9,\"20200331\":9,\"20200401\":9,\"20200402\":9,\"20200403\":8,\"20200404\":8,\"20200405\":8,\"20200406\":8,\"20200407\":8,\"20200408\":8,\"20200409\":8,\"20200410\":8,\"20200411\":8,\"20200412\":7,\"20200413\":7,\"20200414\":7,\"20200415\":7,\"20200416\":7,\"20200417\":6,\"20200418\":6,\"20200419\":6,\"20200420\":6,\"20200421\":6,\"20200422\":6}}],[{\"name\":\"在单比\",\"data\":{\"20200323\":\"0.000%\",\"20200324\":\"11.111%\",\"20200325\":\"11.111%\",\"20200326\":\"11.111%\",\"20200327\":\"11.1